In [1]:
!wget https://pjreddie.com/media/files/yolov3.weights


--2023-12-16 08:54:28--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  20.7MB/s    in 11s     

2023-12-16 08:54:39 (21.0 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
!wget https://github.com/AlexeyAB/darknet/blob/master/cfg/yolov3.cfg


--2023-12-15 09:24:24--  https://github.com/AlexeyAB/darknet/blob/master/cfg/yolov3.cfg
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51609 (50K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]  50.40K  --.-KB/s    in 0.05s   

2023-12-15 09:24:24 (1.01 MB/s) - ‘yolov3.cfg’ saved [51609/51609]



In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import time

def iou(predicted_box, ground_truth_box):
    # Take the intersection and union of the first four values in the tuples
    x1_pred, y1_pred, x2_pred, y2_pred = predicted_box[:4]
    x1_gt, y1_gt, x2_gt, y2_gt = ground_truth_box[:4]

    xA = max(x1_pred, x1_gt)
    yA = max(y1_pred, y1_gt)
    xB = min(x2_pred, x2_gt)
    yB = min(y2_pred, y2_gt)

    intersection_area = max(0, xB - xA) * max(0, yB - yA)
    union_area = ((x2_pred - x1_pred) * (y2_pred - y1_pred)) + ((x2_gt - x1_gt) * (y2_gt - y1_gt)) - intersection_area

    iou = intersection_area / union_area

    # Define a loss function that increases as IoU decreases
    loss = 1.0 - iou  # You can adjust this function as needed

    return loss

# Variables for counting missed detections
total_ground_truth_objects = 0
missed_detection_count = 0

# Variables for FPS calculation
start_time = time.time()
frame_count = 0

# Load YOLOv3 network and configuration
yolov3_net = cv2.dnn.readNet('/content/drive/MyDrive/yolov3.cfg', '/content/yolov3.weights')
# Load YOLOv3 classes (COCO dataset)
classes = []
with open('/content/drive/MyDrive/data1/obj.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Define the target size
target_size = (416, 416)
dataset_dir = '/content/drive/MyDrive/data1'

# Darknet command for YOLOv3
darknet_command = "./darknet detector test /content/drive/MyDrive/data1/obj.data /content/yolov3.cfg /content/yolov3.weights"

# Variables for accumulating loss
total_loss = 0.0
num_iterations = 10000
average_losses = []
iou_values = []

# Iterate through the dataset
for filename in os.listdir(dataset_dir):
    if filename.endswith(".png"):
        image_path = os.path.join(dataset_dir, filename)

        # Construct the path for the associated parameter file (assuming they are in the same directory)
        parameter_file = os.path.splitext(filename)[0] + '.txt'
        parameter_path = os.path.join(dataset_dir, parameter_file)

        if os.path.exists(parameter_path):
            # Load the image
            image = cv2.imread(image_path)

            # Load the parameters from the parameter file
            with open(parameter_path, 'r') as param_file:
                parameters = [line.strip().split() for line in param_file]

            # Prepare the image for YOLOv3
            blob = cv2.dnn.blobFromImage(image, 0.00392, target_size, (0, 0, 0), True, crop=False)
            yolov3_net.setInput(blob)

            # Define the output layer names for YOLOv3
            output_layers = yolov3_net.getUnconnectedOutLayersNames()

            # Run YOLOv3 object detection
            outs = yolov3_net.forward(output_layers)

            # Initialize lists to store prediction boxes and ground truth boxes for the current image
            pred_boxes = []
            gt_boxes = []

            # Iterate through the parameters and draw bounding boxes
            for param in parameters:
                class_id, x, y, width, height = map(float, param)

                # Convert relative coordinates to absolute coordinates
                image_height, image_width, _ = image.shape
                x = int(x * image_width)
                y = int(y * image_height)
                width = int(width * image_width)
                height = int(height * image_height)

                # Calculate the coordinates of the top-left and bottom-right corners
                x1 = x - width // 2
                y1 = y - height // 2
                x2 = x + width // 2
                y2 = y + height // 2

                # Define the ground truth bounding box
                ground_truth_box = (x1, y1, x2, y2)
                gt_boxes.append(ground_truth_box)

                # Print ground truth box values
                print(f"Ground Truth Box: ({x1}, {y1}, {x2}, {y2})")

                # Draw a bounding box
                color = (0, 255, 0)  # Green color
                thickness = 2
                cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

            # Initialize lists to store prediction boxes and ground truth boxes for the current image
            pred_boxes = []
            confidence_threshold = 0.5  # You can adjust this threshold as needed

            # Iterate through the predictions and match them with ground truth
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > confidence_threshold:
                        # Add a check to ensure that class_id is within the valid range
                        if 0 <= class_id < len(classes):
                            box = detection[0:4] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
                            (center_x, center_y, box_width, box_height) = box.astype("int")

                            # Calculate predicted bounding box coordinates
                            x1 = int(center_x - box_width / 2)
                            y1 = int(center_y - box_height / 2)
                            x2 = int(center_x + box_width / 2)
                            y2 = int(center_y + box_height / 2)

                            print(f"Class: {classes[class_id]}, Confidence: {confidence}, Bounding Box: ({x1}, {y1}, {x2}, {y2})")

                            # Append the predicted bounding box to the list
                            pred_boxes.append((x1, y1, x2, y2, confidence))

                            for pred_box in pred_boxes:
                                for gt_box in gt_boxes:
                                    iou_val = iou(pred_box, gt_box)
                                    loss = 1.0 - iou_val
                                    total_loss += loss

                            for pred_box in pred_boxes:
                                for gt_box in gt_boxes:
                                    iou_val = iou(pred_box, gt_box)
                                    iou_values.append(iou_val)

            # Iterate through the predicted boxes and print the coordinates
            for pred_box in pred_boxes:
                x1, y1, x2, y2, confidence = pred_box
                print(f"Predicted Bounding Box: ({x1}, {y1}, {x2}, {y2}), Confidence: {confidence}")

                # If you want to use these values for further processing, you can access them like this:
                # For example, accessing the coordinates of the first predicted box
                x1_first, y1_first, x2_first, y2_first = x1, y1, x2, y2
                confidence_first = confidence
                print(f"Coordinates of the first predicted box: ({x1_first}, {y1_first}, {x2_first}, {y2_first}), Confidence: {confidence_first}")

            else:
                print("No predicted bounding boxes with confidence greater than the threshold.")

            cv2_imshow(image)
           # Save the image with bounding boxes
output_image_path = os.path.join('/content/sample_data/sumit', filename)
cv2.imwrite(output_image_path, image)

            # Calculate and accumulate loss
loss = iou(pred_box, gt_box)
total_loss += loss
num_iterations += 1

            # Calculate average loss after every iteration
avg_loss = total_loss / ((num_iterations + 1) * len(os.listdir(dataset_dir)))
average_losses.append(avg_loss)
print(f'Iteration {num_iterations + 1}, Average Loss: {avg_loss}')

            # Print average loss for every 10 iterations
if num_iterations % 1 == 0:
                avg_loss = total_loss / ((num_iterations + 1) * len(os.listdir(dataset_dir)))
                average_losses.append(avg_loss)
                print(f'Iteration {num_iterations + 1}, Average Loss: {avg_loss}')

            # Count the number of ground truth objects
total_ground_truth_objects += len(gt_boxes)

            # Iterate through the ground truth boxes and check for detection
for gt_box in gt_boxes:
                detection_found = False

                # Check if the ground truth box is detected
                for pred_box in pred_boxes:
                    iou_val = iou(pred_box, gt_box)
                    iou_threshold = 0.5  # You need to define an appropriate IoU threshold
                    if iou_val > iou_threshold:
                        detection_found = True
                        break

                # If the ground truth object is not detected, increment the missed detection count
                if not detection_found:
                    missed_detection_count += 1

# Calculate final average loss
final_avg_loss = total_loss / (num_iterations * len(os.listdir(dataset_dir)))
print(f'Final Average Loss: {final_avg_loss}')

# Calculate and print FPS
end_time = time.time()
elapsed_time = end_time - start_time
fps = frame_count / elapsed_time
print(f'Frames Per Second (FPS): {fps}')

# Calculate missed detection rate
missed_detection_rate = (missed_detection_count / total_ground_truth_objects) * 100
print(f"Missed Detection Rate: {missed_detection_rate}%")

# Calculate average missed detection rate
average_missed_detection_rate = (missed_detection_rate / len(os.listdir(dataset_dir)))
print(f"Average Missed Detection Rate: {average_missed_detection_rate}%")

# Plot the average loss over iterations
plt.plot(average_losses)
plt.xlabel('Iteration')
plt.ylabel('Average Loss')
plt.title('Average Loss Over Iterations')
plt.show()

# Plot the IoU graph
plt.plot(iou_values, marker='o')
plt.title('IoU over Iterations')
plt.xlabel('Iterations')
plt.ylabel('IoU')
plt.show()
# After the loop ends, you can also print a message indicating that the images have been saved
print(f"Images with bounding boxes saved to: /content/drive/MyDrive/output_images/")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
